In [5]:
# import, authenticate and initialize ee
import ee
ee.Initialize()

In [7]:
# import necessary libraries
import geemap

In [8]:
Map = geemap.Map()

In [9]:
ee_path = 'users/balakumaran247/swSentinel/state'

# Polygon Geometry

In [10]:
# state names as in SOI fc
states = {
    "KA": "KARN>TAKA",
    "TN": "TAMIL N>DU",
    }

In [11]:
def filter_geometry(state_abb: str, state_name: str) -> ee.Geometry:
  geometry = ee.FeatureCollection(f'users/balakumaran247/swSentinel/SOI_States')\
              .filter(
                      ee.Filter.eq("STATE", state_name)
                      ).geometry()
  Map.addLayer(geometry, {}, f'{state_abb} Geometry')
  Map.centerObject(geometry)
  return geometry

# Filter Images

In [12]:
def filter_DW(state_abb: str, state_name: str, geometry: ee.Geometry) -> ee.ImageCollection:
  ic = ee.ImageCollection(
      'projects/wri-datalab/dynamic_world/v1/DW_LABELS'
      ).filterBounds(geometry)
  print('start date of the DW filtered collection')
  print(ic.sort(
      'system:time_start',True
      ).first().date().format('Y-M-d').getInfo())
  print('end date of the DW filtered collection')
  print(ic.sort(
      'system:time_start',False
      ).first().date().format('Y-M-d').getInfo())
  kharif = ic.filterDate('2019-06-01', '2019-09-30');
  rabi = ic.filter(ee.Filter.Or(
                ee.Filter.date('2018-12-18', '2019-01-31'),
                ee.Filter.date('2019-10-01', '2019-10-31')));
  zaid = ic.filterDate('2019-02-01', '2019-05-31')
  print(f'{kharif.size().getInfo()} images in Kharif')
  print(f'{rabi.size().getInfo()} images in Rabi')
  print(f'{zaid.size().getInfo()} images in Zaid')
  return ic

## Export Class Image

In [13]:
def generate_class_image(ic: ee.ImageCollection, state_abb: str, geometry: ee.Geometry, class_label: str) -> None:
  if class_label == 'water':
    class_image = ic.mosaic().select(['lulc']).eq(1)
  else:
    class_image = ic.mosaic().select(['lulc']).gt(1)
  class_image = class_image.updateMask(class_image)
  task = ee.batch.Export.image.toAsset(
    image = class_image,
    description = f'{state_abb}_{class_label}_class',
    assetId = f'{ee_path}/{state_abb}_{class_label}_class_image',
    crs = 'EPSG:4326',
    scale = 10,
    region = geometry,
    maxPixels = 1e13
  )
  # task.start()

## Create Filtered Images in EE Assets

In [14]:
for state_abb, state_name in states.items():
  print(state_name)
  geometry = filter_geometry(state_abb, state_name)
  dw_ic = filter_DW(state_abb, state_name, geometry)
  generate_class_image(dw_ic, state_abb, geometry, 'water')
  generate_class_image(dw_ic, state_abb, geometry, 'non_water')

KARN>TAKA
start date of the DW filtered collection
2018-12-18
end date of the DW filtered collection
2019-4-22
0 images in Kharif
4 images in Rabi
17 images in Zaid
TAMIL N>DU
start date of the DW filtered collection
2019-2-23
end date of the DW filtered collection
2019-5-11
0 images in Kharif
0 images in Rabi
25 images in Zaid
ANDHRA PRADESH
start date of the DW filtered collection
2019-1-1
end date of the DW filtered collection
2019-5-26
0 images in Kharif
13 images in Rabi
36 images in Zaid


## Sampling - to be executed after creating filtered images in EE Assets

In [15]:
def sample_class(class_image_path: str, no_points: int) -> ee.FeatureCollection:
  class_image = ee.Image(class_image_path)
  vector = class_image.reduceToVectors(
    reducer = ee.Reducer.countEvery(),
    geometry = class_image.geometry(),
    scale = 10,
    maxPixels = 1e14,
  )
  samples = ee.FeatureCollection.randomPoints(
      region = vector,
      points = no_points,
      seed = 1234,
      maxError = 1)
  return samples

## Export Samples

In [16]:
def export_samples(samples: ee.FeatureCollection, state_abb: str, class_label: str) -> None:
  task = ee.batch.Export.table.toAsset(
    collection = samples,
    description = f'{state_abb}_{class_label}_samples',
    assetId = f'{ee_path}/{state_abb}_{class_label}_samples',
  )
  # task.start()

## Create Samples as FC in GEE Asset

In [17]:
for state_abb, state_name in states.items():
  water_samples = sample_class(f'{ee_path}/{state_abb}_water_class_image', 200)
  export_samples(water_samples, state_abb, 'water')
  non_water_samples = sample_class(f'{ee_path}/{state_abb}_non_water_class_image', 200)
  export_samples(non_water_samples, state_abb, 'non_water')

## Visualize Samples

In [ ]:
def add_to_map(state_abb: str, class_label: str) -> None:
  fc = ee.FeatureCollection(f'{ee_path}/{state_abb}_{class_label}_samples')
  if class_label == 'water':
    color = "blue"
  else:
    color = "red"
  Map.addLayer(fc, {"color": color}, f"{state_abb}_{class_label}_samples")

In [ ]:
for state_abb, state_name in states.items():
  add_to_map(state_abb, 'water')
  add_to_map(state_abb, 'non_water')

In [ ]:
# display the map
Map

Map(center=[15.89546316934291, 80.76101512867518], controls=(WidgetControl(options=['position', 'transparent_b…